<a href="https://colab.research.google.com/github/prakul/MongoDB-AI-Resources/blob/main/Langchain_%2B_MongoDB_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install prerequisites dependencies


In [ ]:
!pip install langchain pypdf pymongo openai python-dotenv tiktoken

# Setup the environment

In [ ]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient

load_dotenv(override=True)

# Add an environment file to the notebook root directory called .env with MONGO_URI="xxx" to load these envvars

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
MONGO_URI = os.environ["MONGO_URI"]
DB_NAME = "langchain-test"
COLLECTION_NAME = "test"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "default"

client = MongoClient(MONGO_URI)
db = client[DB_NAME]
MONGODB_COLLECTION = db[COLLECTION_NAME]

#DATA PREPARATION

 Create an Atlas search index via Atlas UI -> Search -> JSON Editor with the following definition
 ```
 {
   "fields":[
      {
         "type": "vector",
         "path": "embedding",
         "numDimensions": 1536,
         "similarity": "cosine"
      }
   ]
}
```

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("https://arxiv.org/pdf/2303.08774.pdf")
data = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
docs = text_splitter.split_documents(data)

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import MongoDBAtlasVectorSearch

# insert the documents in MongoDB Atlas Vector Search
x = MongoDBAtlasVectorSearch.from_documents(
     documents=docs, embedding=OpenAIEmbeddings(disallowed_special=()), collection=MONGODB_COLLECTION, index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME
 )


#DATA QUERY

In [ ]:

vector_search = MongoDBAtlasVectorSearch.from_connection_string(
    MONGO_URI,
    DB_NAME + "." + COLLECTION_NAME,
    OpenAIEmbeddings(disallowed_special=()),
    index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME
)


In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
question = "How much better is GPT-4 in reducing hallucinations over GPT-3.5"
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


{'query': 'How much better is GPT-4 in reducing hallucinations over GPT-3.5',
 'result': "I'm sorry, but I don't have access to information about the specific capabilities or improvements of GPT-4 over GPT-3.5. My knowledge is based on GPT-3, and I don't have real-time updates on subsequent versions."}

In [ ]:
qa_chain = RetrievalQA.from_chain_type(llm, retriever=vector_search.as_retriever())
qa_chain({"query": question})


{'query': 'How much better is GPT-4 in reducing hallucinations over GPT-3.5',
 'result': 'GPT-4 scores 19 percentage points higher than GPT-3.5 in reducing hallucinations.'}